In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# from subprocess import check_output
# print(check_output(["ls", "./"]).decode("utf8"))

import warnings
warnings.filterwarnings('ignore')


# %config Completer.use_jedi = False
# %load_ext watermark
# %watermark --iversions

In [3]:
fname = "/Users/admin/Desktop/manuscript/"
xl = pd.ExcelFile(fname + 'linear regression data (Recovered).xlsx')
xl.sheet_names

['oviposition data', 'larval lifetables']

## oviposition data

In [26]:
df_ovi = xl.parse(sheet_name='oviposition data')
df_ovi.head()

,habitat_type,month,replicates,temperature,relative_humidity,number_of_larvae
0,tyre,december,1,30.25,73,257
1,tyre,december,2,30.25,73,167
2,tyre,december,3,30.25,73,127
3,tyre,january,1,28.44,78,308
4,tyre,january,2,28.44,78,138


In [27]:
df_ovi.drop(columns='month', axis=1, inplace=True)

In [30]:
df_ovi.columns

Index(['habitat_type ', 'replicates ', 'temperature', 'relative_humidity',
       'number_of_larvae'],
      dtype='object')

In [39]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [38]:
# Creating the DataFrame
data = {
    'habitat_type': ['tyre', 'tyre', 'tyre', 'tyre', 'tyre', 'tyre', 'tyre', 'tyre', 'tyre', 'tyre',
                     'tyre', 'tyre', 'bowls', 'bowls', 'bowls', 'bowls', 'bowls', 'bowls', 'bowls', 'bowls',
                     'bowls', 'bowls', 'bowls', 'bowls', 'drums', 'drums', 'drums', 'drums', 'drums', 'drums',
                     'drums', 'drums', 'drums', 'drums', 'drums', 'drums'],
    'replicates': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
    'temperature': [30.25, 30.25, 30.25, 28.44, 28.44, 28.44, 29.15, 29.15, 29.15, 29.43, 29.43, 29.43,
                    30.25, 30.25, 30.25, 28.44, 28.44, 28.44, 29.15, 29.15, 29.15, 29.43, 28.885, 28.885,
                    30.25, 30.25, 30.25, 28.44, 28.44, 28.44, 29.15, 29.15, 29.15, 29.43, 29.43, 29.43],
    'relative_humidity': [73, 73, 73, 78, 78, 78, 81, 81, 81, 82, 82, 82, 73, 73, 73, 78, 78, 78, 81, 81, 81,
                          82, 82, 82, 73, 73, 73, 78, 78, 78, 81, 81, 81, 82, 82, 82],
    'number_of_larvae': [257, 167, 127, 308, 138, 271, 293, 103, 197, 127, 26, 20, 106, 42, 118, 76, 33, 135,
                         25, 14, 96, 2, 28, 0, 118, 147, 227, 180, 135, 168, 107, 43, 89, 20, 83, 0]
}

df = pd.DataFrame(data)

# Modelling the GLMM
model = smf.mixedlm("number_of_larvae ~ temperature + relative_humidity + replicates", df, groups=df["habitat_type"])
result = model.fit()

# Displaying the summary of the model
print(result.summary())


              Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  number_of_larvae
No. Observations:   36       Method:              REML            
No. Groups:         3        Scale:               3340.2949       
Min. group size:    12       Log-Likelihood:      -185.3614       
Max. group size:    12       Converged:           Yes             
Mean group size:    12.0                                          
------------------------------------------------------------------
                   Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
------------------------------------------------------------------
Intercept         3029.996  705.019  4.298 0.000 1648.186 4411.807
temperature        -56.297   17.831 -3.157 0.002  -91.244  -21.350
relative_humidity  -15.956    3.327 -4.796 0.000  -22.478   -9.435
replicates          -8.403   11.804 -0.712 0.477  -31.540   14.733
Group Var         3225.158   62.632                               



### Interpretation:


The table above delves into the factors influencing the number of mosquito larvae using a `generalized linear mixed model (GLMM)`. This model is particularly powerful because it can simultaneously account for **systematic changes** (fixed effects) and **group-level variations** (random effects) in larval density. Let's explore these concepts alongside the crucial role of **replicates** in strengthening the analysis.

**Fixed Effects and Systematic Changes:**

Fixed effects represent the main factors or variables whose impact on the dependent variable (number of larvae) is being investigated. The GLMM table  shows coefficients, standard errors, z-scores, and p-values for each fixed effect. These values tell us:

* **Direction and Magnitude:** A positive coefficient suggests that a higher value of the fixed effect (e.g., temperature) is linked to an increase in larval density, while a negative coefficient implies a decrease.
* **Variability:** The standard error reflects the variability around the estimated coefficient. A larger standard error indicates less certainty in the estimate.
* **Statistical Significance:** The z-score and p-value together assess the significance of the effect. A low p-value (typically below 0.05) suggests that the observed effect is unlikely due to chance and is statistically significant.

**Random Effects and Group Variation:**

While fixed effects capture systematic changes, the model also considers **random effects**. This component typically focuses on a single value: **Group Variance**. This value captures the variability in larval density **between the distinct groups** (experimental conditions, locations, etc.) included in the study. A larger group variance indicates that the groups have significantly different average larval populations.

**The Importance of Replicates:**

Replicates refer to independent measurements or observations made under the same conditions within a treatment group (or category) in the experiment. Having replicates within each group is crucial for estimating the random effect (group variance) accurately. Why?

* **More Data, Better Estimates:** The more replicates you have in each group, the more data points the model has to understand the variability within that group. This allows for a more precise estimate of the group variance, reflecting the true differences in larval density between the groups.
* **Strengthening Generalizability:** By observing the variability within each group using replicates, we can be more confident that the observed differences between groups are not simply due to chance variation within a single sample. This strengthens the generalizability of the results, meaning the findings are more likely to apply to similar situations beyond the specific groups studied.

**Summary:**

The interplay between fixed effects, random effects, and replicates is essential for a robust interpretation of the GLMM results. Fixed effects highlight the systematic changes associated with specific factors, while random effects capture the variability between groups. Replicates within each group act as building blocks for accurate estimation of the random effect and ultimately strengthen the conclusions drawn from the analysis. 

## larval lifetables

In [4]:
df_lf = xl.parse(sheet_name='larval lifetables')
df_lf.head()

,habitat_type,month,replicates,temperature,relative_humidity,Development_time,Pupation_Rate,Survival_Rate,Emergence_Rate
0,tyre,november,1,27.5,80,6,0.86,0.83,0.96
1,tyre,november,2,27.5,80,8,0.90,0.73,0.81
2,tyre,november,3,27.5,80,7,0.90,0.96,0.77
3,bowls,november,1,27.5,80,9,0.90,70.00,0.77
4,bowls,november,2,27.5,80,9,0.93,80.00,0.85


In [5]:
df_lf.drop(columns='month', axis=1, inplace=True)

In [6]:
df_lf.head()

,habitat_type,replicates,temperature,relative_humidity,Development_time,Pupation_Rate,Survival_Rate,Emergence_Rate
0,tyre,1,27.5,80,6,0.86,0.83,0.96
1,tyre,2,27.5,80,8,0.90,0.73,0.81
2,tyre,3,27.5,80,7,0.90,0.96,0.77
3,bowls,1,27.5,80,9,0.90,70.00,0.77
4,bowls,2,27.5,80,9,0.93,80.00,0.85


In [8]:
# df_lf.to_dict()

In [22]:
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# import pandas as pd

# # Creating the DataFrame
# data = {
#     'habitat_type': ['tyre', 'tyre', 'tyre', 'bowls', 'bowls', 'bowls', 'drums', 'drums', 'drums'],
#     'replicates': [1, 2, 3, 1, 2, 3, 1, 2, 3],
#     'temperature': [27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5, 27.5],
#     'relative_humidity': [80, 80, 80, 80, 80, 80, 80, 80, 80],
#     'Development_time': [6, 8, 7, 9, 9, 9, 13, 11, 14],
#     'Pupation_Rate': [0.86, 0.9, 0.9, 0.9, 0.93, 0.93, 0.93, 0.83, 0.5],
#     'Survival_Rate': [0.83, 0.73, 0.96, 70.0, 80.0, 66.6, 80.0, 73.3, 33.3],
#     'Emergence_Rate': [0.96, 0.81, 0.77, 0.77, 0.85, 0.71, 0.86, 0.88, 0.66]
# }

# df = pd.DataFrame(data)

# # Modelling GLMM for Development_time
# model_dev_time = smf.mixedlm("Development_time ~ temperature + relative_humidity + Pupation_Rate + Survival_Rate + Emergence_Rate", 
#                               df, groups=df["habitat_type"])
# result_dev_time = model_dev_time.fit()

# # Modelling GLMM for Pupation_Rate
# model_pup_rate = smf.mixedlm("Pupation_Rate ~ temperature + relative_humidity + Development_time + Survival_Rate + Emergence_Rate", 
#                               df, groups=df["habitat_type"])
# result_pup_rate = model_pup_rate.fit()

# # Modelling GLMM for Survival_Rate
# model_surv_rate = smf.mixedlm("Survival_Rate ~ temperature + relative_humidity + Development_time + Pupation_Rate + Emergence_Rate", 
#                                df, groups=df["habitat_type"])
# result_surv_rate = model_surv_rate.fit()

# # Modelling GLMM for Emergence_Rate
# model_emerg_rate = smf.mixedlm("Emergence_Rate ~ temperature + relative_humidity + Development_time + Pupation_Rate + Survival_Rate", 
#                                 df, groups=df["habitat_type"])
# result_emerg_rate = model_emerg_rate.fit()

# # Displaying summaries of the models
# print("GLMM for Development_time:")
# print(result_dev_time.summary())
# print("\nAIC:", result_dev_time.aic)
# print("BIC:", result_dev_time.bic)

# print("\nGLMM for Pupation_Rate:")
# print(result_pup_rate.summary())
# print("\nAIC:", result_pup_rate.aic)
# print("BIC:", result_pup_rate.bic)

# print("\nGLMM for Survival_Rate:")
# print(result_surv_rate.summary())
# print("\nAIC:", result_surv_rate.aic)
# print("BIC:", result_surv_rate.bic)

# print("\nGLMM for Emergence_Rate:")
# print(result_emerg_rate.summary())
# print("\nAIC:", result_emerg_rate.aic)
# print("BIC:", result_emerg_rate.bic)


In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

# Creating the DataFrame
data = {
    'habitat_type': ['tyre', 'tyre', 'tyre', 'bowls', 'bowls', 'bowls', 'drums', 'drums', 'drums'],
    'replicates': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'temperature': [26.5, 27.5, 27.5, 25.5, 27.5, 27.5, 24.5, 27.5, 29.5],
    'relative_humidity': [80, 77, 80, 78, 80, 76, 80, 80, 80],
    'Development_time': [6, 8, 7, 9, 9, 9, 13, 11, 14],
    'Pupation_Rate': [0.86, 0.9, 0.9, 0.9, 0.93, 0.93, 0.93, 0.83, 0.5],
    'Survival_Rate': [0.83, 0.73, 0.96, 70.0, 80.0, 66.6, 80.0, 73.3, 33.3],
    'Emergence_Rate': [0.96, 0.81, 0.77, 0.77, 0.85, 0.71, 0.86, 0.88, 0.66]
}

df = pd.DataFrame(data)

# Modelling GLMM for Development_time
model_dev_time = smf.mixedlm("Development_time ~ temperature + relative_humidity + replicates", 
                              df, groups=df["habitat_type"])
result_dev_time = model_dev_time.fit()

# Modelling GLMM for Pupation_Rate
model_pup_rate = smf.mixedlm("Pupation_Rate ~ temperature + relative_humidity + replicates", 
                              df, groups=df["habitat_type"])
result_pup_rate = model_pup_rate.fit()

# Modelling GLMM for Survival_Rate
model_surv_rate = smf.mixedlm("Survival_Rate ~ temperature + relative_humidity + replicates", 
                               df, groups=df["habitat_type"])
result_surv_rate = model_surv_rate.fit()

# Modelling GLMM for Emergence_Rate
model_emerg_rate = smf.mixedlm("Emergence_Rate ~ temperature + relative_humidity + replicates", 
                                df, groups=df["habitat_type"])
result_emerg_rate = model_emerg_rate.fit()

# Displaying summaries of the models
print("GLMM for Development_time:")
print(result_dev_time.summary())
# print("\nAIC:", result_dev_time.aic)
# print("BIC:", result_dev_time.bic)

print("\nGLMM for Pupation_Rate:")
print(result_pup_rate.summary())
# print("\nAIC:", result_pup_rate.aic)
# print("BIC:", result_pup_rate.bic)

print("\nGLMM for Survival_Rate:")
print(result_surv_rate.summary())
# print("\nAIC:", result_surv_rate.aic)
# print("BIC:", result_surv_rate.bic)

print("\nGLMM for Emergence_Rate:")
print(result_emerg_rate.summary())
# print("\nAIC:", result_emerg_rate.aic)
# print("BIC:", result_emerg_rate.bic)


GLMM for Development_time:
            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Development_time
No. Observations: 9       Method:             REML            
No. Groups:       3       Scale:              1.8558          
Min. group size:  3       Log-Likelihood:     -15.4227        
Max. group size:  3       Converged:          Yes             
Mean group size:  3.0                                         
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          17.286   31.154  0.555 0.579 -43.775 78.347
temperature         0.046    0.602  0.077 0.939  -1.133  1.225
relative_humidity  -0.120    0.383 -0.312 0.755  -0.871  0.632
replicates          0.232    1.005  0.231 0.817  -1.738  2.203
Group Var           8.222    8.480                            


GLMM for Pupation_Rate:
           Mixe

/Users/admin/env/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/admin/env/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [20]:
# Calculate correlation matrix
corr_matrix = df[['temperature', 'relative_humidity', 'Pupation_Rate', 'Survival_Rate', 'Emergence_Rate']].corr()

# Display correlation matrix
corr_matrix

,temperature,relative_humidity,Pupation_Rate,Survival_Rate,Emergence_Rate
temperature,1.000000e+00,-9.615340e-16,-0.665454,-0.327126,-0.506720
relative_humidity,-9.615340e-16,1.000000e+00,-0.312186,-0.006314,0.396919
Pupation_Rate,-6.654542e-01,-3.121857e-01,1.000000,0.184363,0.477558
Survival_Rate,-3.271264e-01,-6.314291e-03,0.184363,1.000000,-0.043143
Emergence_Rate,-5.067203e-01,3.969190e-01,0.477558,-0.043143,1.000000
